# Pivot Tables

Hemos visto cómo la abstracción ``GroupBy`` nos permite explorar las relaciones dentro de un conjunto de datos.
Una *pivot tables* es una operación similar que suele verse en hojas de cálculo y otros programas que operan con datos tabulares.
La tabla pivotante toma como entrada datos simples ordenados por columnas y agrupa las entradas en una tabla bidimensional que proporciona un resumen multidimensional de los datos.

Para los ejemplos de esta sección, utilizaremos la base de datos de pasajeros del *Titanic*, disponible a través de la biblioteca Seaborn.

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
titanic = sns.load_dataset('titanic')

In [ ]:
titanic.head()

Contiene abundante información sobre cada pasajero de aquel malogrado viaje: sexo, edad, clase, precio del boleto y mucho más.

## Pivot Tables a mano

Para empezar a aprender más sobre estos datos, podríamos empezar agrupando según el sexo, el estado de supervivencia o alguna combinación de los mismos.
Es posible que sienta la tentación de aplicar una operación ``GroupBy``; por ejemplo, veamos la tasa de supervivencia por sexo:

In [ ]:
titanic.groupby('sex')[['survived']].mean()

Esto nos da inmediatamente una idea: en general, tres de cada cuatro mujeres a bordo sobrevivieron, ¡mientras que sólo uno de cada cinco hombres sobrevivió!

Esto es útil, pero podríamos ir un paso más allá y analizar la supervivencia por sexo y, digamos, por clase.
Usando el vocabulario de ``GroupBy``, podríamos proceder de esta manera:
agrupamos por clase y sexo, seleccionamos la supervivencia, aplicamos un agregado medio, combinamos los grupos resultantes, y luego descomponemos el índice jerárquico para revelar la multidimensionalidad oculta.

In [ ]:
titanic.groupby(['sex', 'class'], observed=False)['survived'].aggregate('mean').unstack()

Esto nos da una mejor idea de cómo el género y la clase afectan a la supervivencia, pero el código empieza a parecer un poco confuso.
Aunque cada paso de este proceso tiene sentido a la luz de las herramientas que hemos discutido anteriormente, la larga cadena de código no es particularmente fácil de leer o utilizar.
Este ``GroupBy`` bidimensional es lo suficientemente común como para que Pandas incluya una rutina de conveniencia, ``pivot_table``, que maneja sucintamente este tipo de agregación multidimensional.

## Sintaxis de las Pivot Table 

Aquí tienes el equivalente a la operación anterior utilizando el método ``pivot_table`` de ``DataFrame``:

In [ ]:
titanic.pivot_table('survived', index='sex', columns='class', observed=False)

Este método es mucho más legible que el enfoque "group by" y produce el mismo resultado.
Como cabría esperar de un crucero transatlántico de principios del siglo XX, el gradiente de supervivencia favorece tanto a las mujeres como a las clases más altas.
Las mujeres de primera clase sobrevivieron con casi total seguridad, mientras que sólo uno de cada diez hombres de tercera clase sobrevivió.

### Pivot Tables Multi-nivel

Al igual que en ``GroupBy``, la agrupación en las tablas pivotes puede especificarse con múltiples niveles, y mediante una serie de opciones.
Por ejemplo, podríamos estar interesados en ver la edad como una tercera dimensión.
Agruparemos la edad utilizando la función ``pd.cut``:

In [ ]:
age = pd.cut(titanic['age'], [0, 18, 80])
titanic.pivot_table('survived', ['sex', age], 'class', observed=False)

También podemos aplicar la misma estrategia al trabajar con las columnas; añadamos información sobre la tarifa pagada utilizando ``pd.qcut`` para calcular automáticamente los cuantiles:

In [ ]:
fare = pd.qcut(titanic['fare'], 2)
titanic.pivot_table('survived', ['sex', age], [fare, 'class'], observed=False)

### Opciones adicionales de Pivot Tables

La firma completa del método ``pivot_table`` de ``DataFrame`` es la siguiente:

```python
DataFrame.pivot_table(data, values=None, index=None, columns=None,
                      aggfunc='mean', fill_value=None, margins=False,
                      dropna=True, margins_name='All')
```

Ya hemos visto ejemplos de los tres primeros argumentos; aquí echaremos un vistazo rápido a los restantes.
Dos de las opciones, ``fill_value`` y ``dropna``, tienen que ver con los datos que faltan y son bastante sencillas; no mostraremos ejemplos de ellas aquí.

La palabra clave ``aggfunc`` controla qué tipo de agregación se aplica, que por defecto es una media.

Como en GroupBy, la especificación de la agregación puede ser una cadena que represente una de las opciones más comunes (por ejemplo, ``sum``, ``mean``, ``count``, ``min``, ``max``, etc.) o una función que implemente una agregación (por ejemplo, ``np.sum()``, ``min()``, ``sum()``, etc.).

Además, se puede especificar como un diccionario que asigna una columna a cualquiera de las opciones deseadas anteriores:

In [ ]:
titanic.pivot_table(index='sex', columns='class',
                    aggfunc={'survived': 'sum', 'fare': 'mean'}, observed=False)


Observa también aquí que hemos omitido la palabra clave ``values``; al especificar una asignación para ``aggfunc``, ésta se determina automáticamente.

A veces es útil calcular totales a lo largo de cada agrupación.
Esto puede hacerse mediante la palabra clave ``margins``:

In [ ]:
titanic.pivot_table('survived', index='sex', columns='class', margins=True, observed=False)

Aquí esto nos da automáticamente información sobre la tasa de supervivencia agnóstica por género, la tasa de supervivencia agnóstica por género por clase, y la tasa de supervivencia global del 38%.

La etiqueta del margen se puede especificar con la palabra clave ``margins_name``, que por defecto es ``"All"``.

## Ejemplo: Datos de natalidad

Como ejemplo más interesante, veamos los datos de libre acceso sobre nacimientos en Estados Unidos, proporcionados por los Centros para el Control de Enfermedades (CDC).
Estos datos pueden consultarse en https://raw.githubusercontent.com/jakevdp/data-CDCbirths/master/births.csv

In [ ]:
# # comando shell para descargar los datos:
# !curl -o data/births.csv https://raw.githubusercontent.com/jakevdp/data-CDCbirths/master/births.csv

In [ ]:
births = pd.read_csv('data/births.csv')

Si echamos un vistazo a los datos, vemos que son relativamente sencillos: contienen el número de nacimientos agrupados por fecha y sexo:

In [ ]:
births.head()

Podemos empezar a entender estos datos un poco mejor utilizando una tabla dinámica.
Añadamos una columna de década y veamos los nacimientos de hombres y mujeres en función de la década:

In [ ]:
births['decade'] = 10 * (births['year'] // 10)
births.pivot_table('births', index='decade', columns='gender', aggfunc='sum')

Inmediatamente vemos que los nacimientos masculinos superan a los femeninos en cada década.
Para ver esta tendencia un poco más claramente, podemos utilizar las herramientas de trazado incorporadas en Pandas para visualizar el número total de nacimientos por año.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style="whitegrid")

births.pivot_table('births', index='year', columns='gender', aggfunc='sum').plot()
plt.ylabel('total births per year');

Con una simple tabla dinámica y el método ``plot()``, podemos ver inmediatamente la tendencia anual de los nacimientos por sexo. A simple vista, parece que en los últimos 50 años los nacimientos masculinos han superado a los femeninos en un 5% aproximadamente.

### Mayor exploración de datos

Aunque esto no está necesariamente relacionado con la pivot table, hay algunas características más interesantes que podemos extraer de este conjunto de datos utilizando las herramientas de Pandas que hemos visto hasta ahora.
Debemos empezar por limpiar un poco los datos, eliminando los valores atípicos causados por fechas mal escritas (por ejemplo, 31 de junio) o valores que faltan (por ejemplo, 99 de junio).
Una forma sencilla de eliminarlos todos a la vez es cortar los valores atípicos; lo haremos mediante una operación robusta de recorte sigma:

In [ ]:
quartiles = np.percentile(births['births'], [25, 50, 75])
mu = quartiles[1]
sig = 0.74 * (quartiles[2] - quartiles[0])

Esta línea final es una estimación robusta de la media muestral, donde el 0,74 procede del rango intercuartílico de una distribución gaussiana.

Con esto podemos utilizar el método ``query()`` para filtrar las filas con nacimientos fuera de estos valores:

In [ ]:
births = births.query('(births > @mu - 5 * @sig) & (births < @mu + 5 * @sig)')

A continuación, cambiamos la columna ``day`` a números enteros; antes era una cadena porque algunas columnas del conjunto de datos contenían el valor ``'null'``:

In [ ]:
# cambiar la columna 'day' a entero; originalmente era una cadena debido a los nulos
births['day'] = births['day'].astype(int)

Por último, podemos combinar el día, el mes y el año para crear un índice Fecha.

Esto nos permite calcular rápidamente el día de la semana correspondiente a cada fila:

In [ ]:
# crear un índice datetime a partir del año, mes, día
births.index = pd.to_datetime(10000 * births.year +
                              100 * births.month +
                              births.day, format='%Y%m%d')

births['dayofweek'] = births.index.dayofweek

De este modo, podemos trazar los nacimientos por día de la semana durante varias décadas:

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl

births.pivot_table('births', index='dayofweek', columns='decade', aggfunc='mean').plot()
ax = plt.gca()

ax.set_xticks(range(7))
ax.set_xticklabels(['Mon', 'Tues', 'Wed', 'Thurs', 'Fri', 'Sat', 'Sun'])

plt.ylabel('Media de nacimientos por día')
plt.show()


Al parecer, los nacimientos son ligeramente menos frecuentes los fines de semana que entre semana. Nótese que faltan los años 90 y 2000 porque los datos de los CDC sólo contienen el mes de nacimiento a partir de 1989.

Otra visión interesante es representar el número medio de nacimientos por día del *año*.
Agrupemos primero los datos por mes y día por separado:

In [ ]:
births_by_date = births.pivot_table('births', 
                                    [births.index.month, births.index.day])
births_by_date.head()

El resultado es un multiíndice de meses y días.
Para que sea más fácil de trazar, convirtamos estos meses y días en una fecha asociándolos a una variable de año ficticia (¡asegurándonos de elegir un año bisiesto para que el 29 de febrero se trate correctamente!)

In [ ]:
births_by_date.index = [pd.Timestamp(2012, month, day)
                        for (month, day) in births_by_date.index]
births_by_date.head()

Centrándonos sólo en el mes y el día, ahora tenemos una serie temporal que refleja el número medio de nacimientos por fecha del año.
A partir de aquí, podemos utilizar el método ``plot`` para representar gráficamente los datos. Esto revela algunas tendencias interesantes:

In [ ]:
# Trazar los resultados
fig, ax = plt.subplots(figsize=(12, 4))
births_by_date.plot(ax=ax);

En concreto, lo más llamativo de este gráfico es la caída de la natalidad en los días festivos de EE.UU. (por ejemplo, el Día de la Independencia, el Día del Trabajo, Acción de Gracias, Navidad, Año Nuevo), aunque es probable que esto refleje tendencias en los nacimientos programados/inducidos más que algún profundo efecto psicosomático sobre los nacimientos naturales.

Si observamos este breve ejemplo, veremos que muchas de las herramientas de Python y Pandas que hemos visto hasta ahora pueden combinarse y utilizarse para obtener información de una gran variedad de conjuntos de datos.

<!--NAVIGATION-->
< [Agregación y agrupación](7-Agregaciones_y_agrupaciones.ipynb) | [Trabajar con string](9-Trabajar_con_string.ipynb) >